In [1]:
import json
from pyfuseki import FusekiUpdate
import httpx
from pysolr import Solr
from rdflib import Graph

In [4]:
graph = Graph()
graph.parse('http://id.loc.gov/authorities/names/n80002329')

<Graph identifier=Nd07c446e9b10488480d7067d3c50414c (<class 'rdflib.graph.Graph'>)>

In [5]:
graph.serialize('machado.ttl')

<Graph identifier=Nd07c446e9b10488480d7067d3c50414c (<class 'rdflib.graph.Graph'>)>

In [3]:
solr = Solr('http://localhost:8983/solr/authority/', timeout=10)

In [2]:
upAcervo = FusekiUpdate('http://localhost:3030', 'authority')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response.convert()

solrAuthority = Solr('http://localhost:8983/solr/authority/', timeout=10)
solrAuthority.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">122</int>\n</lst>\n</response>\n'

In [3]:
with open("agents.json", encoding="utf-8") as file:
    reader = file.read()
    agents = json.loads(reader)
    file.close()

In [8]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = agents[0]
response = httpx.post('http://localhost:8000/authorities/agents/', headers=headers, json=json_data)
response

<Response [201 Created]>

In [4]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for agent in agents:
    response = httpx.post('http://localhost:8000/authorities/agents/', headers=headers, json=agent, timeout=1000)
    print(response.json())


{'id': 'bka-1', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":294\n  }\n}'}
{'id': 'bka-2', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":136\n  }\n}'}
{'id': 'bka-3', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":143\n  }\n}'}
{'id': 'bka-4', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":157\n  }\n}'}
{'id': 'bka-5', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":193\n  }\n}'}
{'id': 'bka-6', 'jena': 'Update succeeded', 'solr': '{\n  "responseHeader":{\n    "status":0,\n    "QTime":194\n  }\n}'}


In [5]:
from pydantic import BaseModel, Field
from typing import Optional
from api.src.schemas.authorities.authority import Variant, AdminMetadata, Element
from datetime import datetime, date

class Uri(BaseModel):
    value: str
    label: str
    base: Optional[str] = None

class Affiliation(BaseModel):
    organization: Uri
    affiliationStart: Optional[str] = None
    affiliationEnd: Optional[str] = None

class Status(BaseModel):
    value: str = Field(default="mstatus:new")
    label: str = Field(default="novo")
    
class AdminMetadata(BaseModel):
    assigner: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    descriptionModifier: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    creationDate: date = Field(default=date.today())
    descriptionLanguage: str = Field(default="http://id.loc.gov/vocabulary/languages/por")
    generationProcess: str = Field(default="BiblioKeia v.1")
    generationDate: str = Field(default=datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))
    # identifiedBy: list[IdentifiedBy]
    status: Status = Field(default=Status(value="mstatus:new", label="novo"))
    
class Agents(BaseModel):
    type: str 
    identifiersLccn: Optional[str] = None
    identifiersLocal: int
    adminMetadata: AdminMetadata 
    authoritativeLabel: str
    elementList: list[Element]
    fullerName: Optional[Element] = None
    birthDate: Optional[str] = None
    birthPlace: Optional[str] = None
    deathDate: Optional[str] = None
    hasAffiliation: Optional[list[Affiliation]] = None
    occupation: Optional[list[Uri]] = None
    fieldOfActivity: Optional[list[Uri]] = None
    hasCloseExternalAuthority: Optional[list[Uri]] = None
    hasExactExternalAuthority: Optional[list[Uri]] = None
    hasVariant: Optional[list[Variant]] = None
    subjectOf: Optional[list[Uri]] = None
    contributorOf: Optional[list[Uri]] = None
    isMemberOfMADSCollection: str

In [17]:
a = {
    "type": "PersonalName",
    "identifiersLocal": 1,
    "adminMetadata": {
        "creationDate": "2023-09-19",
    },
    "authoritativeLabel": "Machado de Assis",
    "elementList": [
        {
      "type": "FullNameElement",
      "elementValue": {
        "value": "Machado de Assis"
      }
    },
    {
      "type": "DateNameElement",
      "elementValue": {
        "value": "1839-1908"
      }
    } ],
    "isMemberOfMADSCollection": "http://bibliokeia.com/authorities/PersonalName/"
}

agent = Agents(**a)



In [20]:
agent.identifiersLccn

In [15]:
agent.model_dump()

{'type': 'PersonalName',
 'identifiersLccn': None,
 'identifiersLocal': 1,
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 9, 19),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-09-19T13:32:50',
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'authoritativeLabel': 'Machado de Assis',
 'elementList': [{'type': 'FullNameElement',
   'elementValue': {'value': 'Machado de Assis', 'lang': None}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1839-1908', 'lang': None}}],
 'fullerName': None,
 'birthDate': None,
 'birthPlace': None,
 'deathDate': None,
 'hasAffiliation': None,
 'occupation': None,
 'fieldOfActivity': None,
 'hasCloseExternalAuthority': None,
 'hasExactExternalAuthority': None,
 'hasVariant': None,
 'subjectOf':

In [18]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

response = httpx.post('http://localhost:8000/authorities/agents/', headers=headers, json=a, timeout=1000)
print(response.json())

{'detail': [{'type': 'missing', 'loc': ['body', 'adminMetadata', 'identifiedBy'], 'msg': 'Field required', 'input': {'creationDate': '2023-09-19'}, 'url': 'https://errors.pydantic.dev/2.0.3/v/missing'}]}
